In [2]:
import sys
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras import regularizers
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support

%matplotlib inline

print keras.__version__

Using TensorFlow backend.


2.0.8


# 'whole', already trained in 'tactile-training' notebook

# 'laying' train, 'standing' test

In [6]:
train_tactile_images = np.load('images-laying-t3-cero2lesscontact-stdnorm.npy')
train_labels = np.load('labels-laying-t3-cero2lesscontact-stdnorm.npy')
train_labels_cat = np_utils.to_categorical(train_labels, num_classes=2)

test_tactile_images = np.load('images-standing-t3-cero2lesscontact-stdnorm.npy')
test_labels = np.load('labels-standing-t3-cero2lesscontact-stdnorm.npy')
test_labels_cat = np_utils.to_categorical(test_labels, num_classes=2)

FINGERS = 3
TACTILE_IMAGE_ROWS = 6
TACTILE_IMAGE_COLS = 7

In [8]:
# BEST

folds = 10
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for fold in range(folds):
    print "Training fold " + str(fold + 1) + "/" + str(folds)
    
    shuffle(train_tactile_images, train_labels_cat)
    shuffle(test_tactile_images, test_labels_cat)
    
    # build model
    epochs = 200
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.005
    
    l2_reg = 0.01
    drop_prob = 0.0
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(train_tactile_images, train_labels_cat, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print "# # # Evaluating cv-fold..."
    scores = model.evaluate(test_tactile_images, test_labels_cat, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(test_tactile_images)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(test_labels_cat, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print "\naccuracy:", scores[1] * 100
    print "precision:", precision * 100
    print "recall:", recall * 100
    print "f1_score:", f1_score * 100
    
print "\nFinal cross-validation score:"
print np.mean(cv_accuracies), "+/-", np.std(cv_accuracies)
print np.mean(cv_precision), "+/-", np.std(cv_precision)
print np.mean(cv_recall), "+/-", np.std(cv_recall)
print np.mean(cv_f1_score), "+/-", np.std(cv_f1_score)

Training fold 1/10
Epoch 1/200
1276/1276 [==============================] - 0s - loss: 8.0086 - acc: 0.8096     
Epoch 2/200
1276/1276 [==============================] - 0s - loss: 7.1878 - acc: 0.8840     
Epoch 3/200
1276/1276 [==============================] - 0s - loss: 6.6060 - acc: 0.9060     
Epoch 4/200
1276/1276 [==============================] - 0s - loss: 6.1556 - acc: 0.9154     
Epoch 5/200
1276/1276 [==============================] - 0s - loss: 5.7796 - acc: 0.9169     
Epoch 6/200
1276/1276 [==============================] - 0s - loss: 5.4563 - acc: 0.9303     
Epoch 7/200
1276/1276 [==============================] - 0s - loss: 5.1808 - acc: 0.9389     
Epoch 8/200
1276/1276 [==============================] - 0s - loss: 4.9434 - acc: 0.9412     
Epoch 9/200
1276/1276 [==============================] - 0s - loss: 4.7278 - acc: 0.9420     
Epoch 10/200
1276/1276 [==============================] - 0s - loss: 4.5566 - acc: 0.9459     
Epoch 11/200
1276/1276 [================

1276/1276 [==============================] - 0s - loss: 1.7817 - acc: 0.9812     
Epoch 88/200
1276/1276 [==============================] - 0s - loss: 1.7721 - acc: 0.9781     
Epoch 89/200
1276/1276 [==============================] - 0s - loss: 1.7729 - acc: 0.9757     
Epoch 90/200
1276/1276 [==============================] - 0s - loss: 1.7520 - acc: 0.9812     
Epoch 91/200
1276/1276 [==============================] - 0s - loss: 1.7460 - acc: 0.9820     
Epoch 92/200
1276/1276 [==============================] - 0s - loss: 1.7439 - acc: 0.9773     
Epoch 93/200
1276/1276 [==============================] - 0s - loss: 1.7307 - acc: 0.9828     
Epoch 94/200
1276/1276 [==============================] - 0s - loss: 1.7224 - acc: 0.9812     
Epoch 95/200
1276/1276 [==============================] - 0s - loss: 1.7210 - acc: 0.9765     
Epoch 96/200
1276/1276 [==============================] - 0s - loss: 1.7067 - acc: 0.9773     
Epoch 97/200
1276/1276 [==============================] - 0s - 

1276/1276 [==============================] - 0s - loss: 2.1541 - acc: 0.9741     
Epoch 57/200
1276/1276 [==============================] - 0s - loss: 2.1469 - acc: 0.9663     
Epoch 58/200
1276/1276 [==============================] - 0s - loss: 2.1276 - acc: 0.9741     
Epoch 59/200
1276/1276 [==============================] - 0s - loss: 2.1082 - acc: 0.9726     
Epoch 60/200
1276/1276 [==============================] - 0s - loss: 2.0982 - acc: 0.9671     
Epoch 61/200
1276/1276 [==============================] - 0s - loss: 2.0724 - acc: 0.9757     
Epoch 62/200
1276/1276 [==============================] - 0s - loss: 2.0655 - acc: 0.9749     
Epoch 63/200
1276/1276 [==============================] - 0s - loss: 2.0429 - acc: 0.9773     
Epoch 64/200
1276/1276 [==============================] - 0s - loss: 2.0293 - acc: 0.9773     
Epoch 65/200
1276/1276 [==============================] - 0s - loss: 2.0245 - acc: 0.9749     
Epoch 66/200
1276/1276 [==============================] - 0s - 

1276/1276 [==============================] - 0s - loss: 3.1933 - acc: 0.9624     
Epoch 25/200
1276/1276 [==============================] - 0s - loss: 3.1298 - acc: 0.9624     
Epoch 26/200
1276/1276 [==============================] - 0s - loss: 3.0772 - acc: 0.9592     
Epoch 27/200
1276/1276 [==============================] - 0s - loss: 3.0230 - acc: 0.9655     
Epoch 28/200
1276/1276 [==============================] - 0s - loss: 2.9749 - acc: 0.9655     
Epoch 29/200
1276/1276 [==============================] - 0s - loss: 2.9305 - acc: 0.9655     
Epoch 30/200
1276/1276 [==============================] - 0s - loss: 2.8838 - acc: 0.9702     
Epoch 31/200
1276/1276 [==============================] - 0s - loss: 2.8561 - acc: 0.9561     
Epoch 32/200
1276/1276 [==============================] - 0s - loss: 2.8062 - acc: 0.9639     
Epoch 33/200
1276/1276 [==============================] - 0s - loss: 2.7602 - acc: 0.9702     
Epoch 34/200
1276/1276 [==============================] - 0s - 

1276/1276 [==============================] - 0s - loss: 1.3099 - acc: 0.9890     
Epoch 196/200
1276/1276 [==============================] - 0s - loss: 1.3179 - acc: 0.9796     
Epoch 197/200
1276/1276 [==============================] - 0s - loss: 1.3107 - acc: 0.9851     
Epoch 198/200
1276/1276 [==============================] - 0s - loss: 1.3113 - acc: 0.9859     
Epoch 199/200
1276/1276 [==============================] - 0s - loss: 1.2962 - acc: 0.9898     
Epoch 200/200
1276/1276 [==============================] - 0s - loss: 1.3012 - acc: 0.9875     
# # # Evaluating cv-fold...
1120/1273 [=========================>....] - ETA: 0s
accuracy: 64.1005498634
precision: 69.1202872531
recall: 57.4626865672
f1_score: 62.7546862266
Training fold 4/10
Epoch 1/200
1276/1276 [==============================] - 0s - loss: 8.0256 - acc: 0.8150     
Epoch 2/200
1276/1276 [==============================] - 0s - loss: 7.2500 - acc: 0.8801     
Epoch 3/200
1276/1276 [==============================] 

...

1276/1276 [==============================] - 0s - loss: 1.9578 - acc: 0.9757     
Epoch 71/200
1276/1276 [==============================] - 0s - loss: 1.9559 - acc: 0.9749     
Epoch 72/200
1276/1276 [==============================] - 0s - loss: 1.9352 - acc: 0.9734     
Epoch 73/200
1276/1276 [==============================] - 0s - loss: 1.9277 - acc: 0.9773     
Epoch 74/200
1276/1276 [==============================] - 0s - loss: 1.9197 - acc: 0.9757     
Epoch 75/200
1276/1276 [==============================] - 0s - loss: 1.9047 - acc: 0.9726     
Epoch 76/200
1276/1276 [==============================] - 0s - loss: 1.8922 - acc: 0.9781     
Epoch 77/200
1276/1276 [==============================] - 0s - loss: 1.8810 - acc: 0.9765     
Epoch 78/200
1276/1276 [==============================] - 0s - loss: 1.8644 - acc: 0.9804     
Epoch 79/200
1276/1276 [==============================] - 0s - loss: 1.8514 - acc: 0.9812     
Epoch 80/200
1276/1276 [==============================] - 0s - 

1276/1276 [==============================] - 0s - loss: 2.4580 - acc: 0.9710     
Epoch 39/200
1276/1276 [==============================] - 0s - loss: 2.4376 - acc: 0.9710     
Epoch 40/200
1276/1276 [==============================] - 0s - loss: 2.4105 - acc: 0.9694     
Epoch 41/200
1276/1276 [==============================] - 0s - loss: 2.3754 - acc: 0.9734     
Epoch 42/200
1276/1276 [==============================] - 0s - loss: 2.3595 - acc: 0.9694     
Epoch 43/200
1276/1276 [==============================] - 0s - loss: 2.3224 - acc: 0.9749     
Epoch 44/200
1276/1276 [==============================] - 0s - loss: 2.3060 - acc: 0.9702     
Epoch 45/200
1276/1276 [==============================] - 0s - loss: 2.2809 - acc: 0.9765     
Epoch 46/200
1276/1276 [==============================] - 0s - loss: 2.2544 - acc: 0.9718     
Epoch 47/200
1276/1276 [==============================] - 0s - loss: 2.2424 - acc: 0.9663     
Epoch 48/200
1276/1276 [==============================] - 0s - 

1276/1276 [==============================] - 0s - loss: 1.4775 - acc: 0.9820     
Epoch 125/200
1276/1276 [==============================] - 0s - loss: 1.4692 - acc: 0.9851     
Epoch 126/200
1276/1276 [==============================] - 0s - loss: 1.4711 - acc: 0.9843     
Epoch 127/200
1276/1276 [==============================] - 0s - loss: 1.4657 - acc: 0.9828     
Epoch 128/200
1276/1276 [==============================] - 0s - loss: 1.4581 - acc: 0.9820     
Epoch 129/200
1276/1276 [==============================] - 0s - loss: 1.4591 - acc: 0.9828     
Epoch 130/200
1276/1276 [==============================] - 0s - loss: 1.4494 - acc: 0.9851     
Epoch 131/200
1276/1276 [==============================] - 0s - loss: 1.4520 - acc: 0.9773     
Epoch 132/200
1276/1276 [==============================] - 0s - loss: 1.4456 - acc: 0.9851     
Epoch 133/200
1276/1276 [==============================] - 0s - loss: 1.4440 - acc: 0.9788     
Epoch 134/200
1276/1276 [=============================

1276/1276 [==============================] - 0s - loss: 1.7294 - acc: 0.9820     
Epoch 94/200
1276/1276 [==============================] - 0s - loss: 1.7205 - acc: 0.9812     
Epoch 95/200
1276/1276 [==============================] - 0s - loss: 1.7063 - acc: 0.9812     
Epoch 96/200
1276/1276 [==============================] - 0s - loss: 1.7021 - acc: 0.9804     
Epoch 97/200
1276/1276 [==============================] - 0s - loss: 1.6971 - acc: 0.9796     
Epoch 98/200
1276/1276 [==============================] - 0s - loss: 1.6852 - acc: 0.9828     
Epoch 99/200
1276/1276 [==============================] - 0s - loss: 1.6781 - acc: 0.9812     
Epoch 100/200
1276/1276 [==============================] - 0s - loss: 1.6810 - acc: 0.9773     
Epoch 101/200
1276/1276 [==============================] - 0s - loss: 1.6724 - acc: 0.9788     
Epoch 102/200
1276/1276 [==============================] - 0s - loss: 1.6687 - acc: 0.9796     
Epoch 103/200
1276/1276 [==============================] - 0

1276/1276 [==============================] - 0s - loss: 2.0486 - acc: 0.9796     
Epoch 63/200
1276/1276 [==============================] - 0s - loss: 2.0343 - acc: 0.9773     
Epoch 64/200
1276/1276 [==============================] - 0s - loss: 2.0289 - acc: 0.9687     
Epoch 65/200
1276/1276 [==============================] - 0s - loss: 2.0031 - acc: 0.9773     
Epoch 66/200
1276/1276 [==============================] - 0s - loss: 1.9909 - acc: 0.9749     
Epoch 67/200
1276/1276 [==============================] - 0s - loss: 1.9911 - acc: 0.9702     
Epoch 68/200
1276/1276 [==============================] - 0s - loss: 1.9679 - acc: 0.9773     
Epoch 69/200
1276/1276 [==============================] - 0s - loss: 1.9605 - acc: 0.9741     
Epoch 70/200
1276/1276 [==============================] - 0s - loss: 1.9500 - acc: 0.9734     
Epoch 71/200
1276/1276 [==============================] - 0s - loss: 1.9278 - acc: 0.9820     
Epoch 72/200
1276/1276 [==============================] - 0s - 

# 'standing' train, 'laying' test

In [15]:
train_tactile_images = np.load('images-standing-t3-cero2lesscontact-stdnorm.npy')
train_labels = np.load('labels-standing-t3-cero2lesscontact-stdnorm.npy')
train_labels_cat = np_utils.to_categorical(train_labels, num_classes=2)

test_tactile_images = np.load('images-laying-t3-cero2lesscontact-stdnorm.npy')
test_labels = np.load('labels-laying-t3-cero2lesscontact-stdnorm.npy')
test_labels_cat = np_utils.to_categorical(test_labels, num_classes=2)

FINGERS = 3
TACTILE_IMAGE_ROWS = 6
TACTILE_IMAGE_COLS = 7

In [16]:
# BEST

folds = 10
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for fold in range(folds):
    print "Training fold " + str(fold + 1) + "/" + str(folds)
    
    shuffle(train_tactile_images, train_labels_cat)
    shuffle(test_tactile_images, test_labels_cat)
    
    # build model
    epochs = 200
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.007
    
    l2_reg = 0.01
    drop_prob = 0.2
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(train_tactile_images, train_labels_cat, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print "# # # Evaluating cv-fold..."
    scores = model.evaluate(test_tactile_images, test_labels_cat, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(test_tactile_images)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(test_labels_cat, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print "\naccuracy:", scores[1] * 100
    print "precision:", precision * 100
    print "recall:", recall * 100
    print "f1_score:", f1_score * 100
    
print "\nFinal cross-validation score:"
print np.mean(cv_accuracies), "+/-", np.std(cv_accuracies)
print np.mean(cv_precision), "+/-", np.std(cv_precision)
print np.mean(cv_recall), "+/-", np.std(cv_recall)
print np.mean(cv_f1_score), "+/-", np.std(cv_f1_score)

Training fold 1/10
Epoch 1/200
1273/1273 [==============================] - 1s - loss: 8.0383 - acc: 0.8068     
Epoch 2/200
1273/1273 [==============================] - 0s - loss: 7.2833 - acc: 0.9010     
Epoch 3/200
1273/1273 [==============================] - 0s - loss: 6.8076 - acc: 0.9026     
Epoch 4/200
1273/1273 [==============================] - 0s - loss: 6.4007 - acc: 0.9159     
Epoch 5/200
1273/1273 [==============================] - 0s - loss: 6.1019 - acc: 0.9183     
Epoch 6/200
1273/1273 [==============================] - 0s - loss: 5.8119 - acc: 0.9379     
Epoch 7/200
1273/1273 [==============================] - 0s - loss: 5.5988 - acc: 0.9277     
Epoch 8/200
1273/1273 [==============================] - 0s - loss: 5.3908 - acc: 0.9372     
Epoch 9/200
1273/1273 [==============================] - 0s - loss: 5.2123 - acc: 0.9340     
Epoch 10/200
1273/1273 [==============================] - 0s - loss: 5.0714 - acc: 0.9387     
Epoch 11/200
1273/1273 [================

1273/1273 [==============================] - 0s - loss: 2.4390 - acc: 0.9796     
Epoch 88/200
1273/1273 [==============================] - 0s - loss: 2.4413 - acc: 0.9678     
Epoch 89/200
1273/1273 [==============================] - 0s - loss: 2.4224 - acc: 0.9780     
Epoch 90/200
1273/1273 [==============================] - 0s - loss: 2.4101 - acc: 0.9772     
Epoch 91/200
1273/1273 [==============================] - 0s - loss: 2.3961 - acc: 0.9796     
Epoch 92/200
1273/1273 [==============================] - 0s - loss: 2.4005 - acc: 0.9725     
Epoch 93/200
1273/1273 [==============================] - 0s - loss: 2.3871 - acc: 0.9780     
Epoch 94/200
1273/1273 [==============================] - 0s - loss: 2.3752 - acc: 0.9819     
Epoch 95/200
1273/1273 [==============================] - 0s - loss: 2.3724 - acc: 0.9827     
Epoch 96/200
1273/1273 [==============================] - 0s - loss: 2.3542 - acc: 0.9811     
Epoch 97/200
1273/1273 [==============================] - 0s - 

1273/1273 [==============================] - 0s - loss: 2.7566 - acc: 0.9694     
Epoch 57/200
1273/1273 [==============================] - 0s - loss: 2.7379 - acc: 0.9654     
Epoch 58/200
1273/1273 [==============================] - 0s - loss: 2.7165 - acc: 0.9741     
Epoch 59/200
1273/1273 [==============================] - 0s - loss: 2.7004 - acc: 0.9733     
Epoch 60/200
1273/1273 [==============================] - 0s - loss: 2.6872 - acc: 0.9694     
Epoch 61/200
1273/1273 [==============================] - 0s - loss: 2.6692 - acc: 0.9701     
Epoch 62/200
1273/1273 [==============================] - 0s - loss: 2.6541 - acc: 0.9701     
Epoch 63/200
1273/1273 [==============================] - 0s - loss: 2.6341 - acc: 0.9725     
Epoch 64/200
1273/1273 [==============================] - 0s - loss: 2.6346 - acc: 0.9678     
Epoch 65/200
1273/1273 [==============================] - 0s - loss: 2.6065 - acc: 0.9764     
Epoch 66/200
1273/1273 [==============================] - 0s - 

1273/1273 [==============================] - 0s - loss: 3.8036 - acc: 0.9607     
Epoch 25/200
1273/1273 [==============================] - 0s - loss: 3.7374 - acc: 0.9662     
Epoch 26/200
1273/1273 [==============================] - 0s - loss: 3.6967 - acc: 0.9568     
Epoch 27/200
1273/1273 [==============================] - 0s - loss: 3.6409 - acc: 0.9576     
Epoch 28/200
1273/1273 [==============================] - 0s - loss: 3.5916 - acc: 0.9639     
Epoch 29/200
1273/1273 [==============================] - 0s - loss: 3.5477 - acc: 0.9694     
Epoch 30/200
1273/1273 [==============================] - 0s - loss: 3.5114 - acc: 0.9647     
Epoch 31/200
1273/1273 [==============================] - 0s - loss: 3.4743 - acc: 0.9662     
Epoch 32/200
1273/1273 [==============================] - 0s - loss: 3.4383 - acc: 0.9592     
Epoch 33/200
1273/1273 [==============================] - 0s - loss: 3.3926 - acc: 0.9647     
Epoch 34/200
1273/1273 [==============================] - 0s - 

1273/1273 [==============================] - 0s - loss: 1.8167 - acc: 0.9882     
Epoch 196/200
1273/1273 [==============================] - 0s - loss: 1.8131 - acc: 0.9859     
Epoch 197/200
1273/1273 [==============================] - 0s - loss: 1.8149 - acc: 0.9866     
Epoch 198/200
1273/1273 [==============================] - 0s - loss: 1.8058 - acc: 0.9882     
Epoch 199/200
1273/1273 [==============================] - 0s - loss: 1.8053 - acc: 0.9859     
Epoch 200/200
1273/1273 [==============================] - 0s - loss: 1.8048 - acc: 0.9835     
# # # Evaluating cv-fold...
 928/1276 [====================>.........] - ETA: 0s 
accuracy: 67.0062695925
precision: 92.7272727273
recall: 33.4975369458
f1_score: 49.2159227986
Training fold 4/10
Epoch 1/200
1273/1273 [==============================] - 1s - loss: 8.0945 - acc: 0.7973     
Epoch 2/200
1273/1273 [==============================] - 0s - loss: 7.2890 - acc: 0.9089     
Epoch 3/200
1273/1273 [==============================]

...

1273/1273 [==============================] - 0s - loss: 2.5358 - acc: 0.9788     
Epoch 73/200
1273/1273 [==============================] - 0s - loss: 2.5334 - acc: 0.9686     
Epoch 74/200
1273/1273 [==============================] - 0s - loss: 2.5193 - acc: 0.9756     
Epoch 75/200
1273/1273 [==============================] - 0s - loss: 2.5094 - acc: 0.9756     
Epoch 76/200
1273/1273 [==============================] - 0s - loss: 2.4849 - acc: 0.9827     
Epoch 77/200
1273/1273 [==============================] - 0s - loss: 2.4783 - acc: 0.9780     
Epoch 78/200
1273/1273 [==============================] - 0s - loss: 2.4831 - acc: 0.9709     
Epoch 79/200
1273/1273 [==============================] - 0s - loss: 2.4570 - acc: 0.9780     
Epoch 80/200
1273/1273 [==============================] - 0s - loss: 2.4567 - acc: 0.9756     
Epoch 81/200
1273/1273 [==============================] - 0s - loss: 2.4286 - acc: 0.9796     
Epoch 82/200
1273/1273 [==============================] - 0s - 

1273/1273 [==============================] - 0s - loss: 3.1018 - acc: 0.9709     
Epoch 41/200
1273/1273 [==============================] - 0s - loss: 3.0778 - acc: 0.9678     
Epoch 42/200
1273/1273 [==============================] - 0s - loss: 3.0453 - acc: 0.9701     
Epoch 43/200
1273/1273 [==============================] - 0s - loss: 3.0267 - acc: 0.9701     
Epoch 44/200
1273/1273 [==============================] - 0s - loss: 2.9954 - acc: 0.9709     
Epoch 45/200
1273/1273 [==============================] - 0s - loss: 2.9841 - acc: 0.9607     
Epoch 46/200
1273/1273 [==============================] - 0s - loss: 2.9482 - acc: 0.9686     
Epoch 47/200
1273/1273 [==============================] - 0s - loss: 2.9228 - acc: 0.9717     
Epoch 48/200
1273/1273 [==============================] - 0s - loss: 2.9062 - acc: 0.9694     
Epoch 49/200
1273/1273 [==============================] - 0s - loss: 2.8795 - acc: 0.9741     
Epoch 50/200
1273/1273 [==============================] - 0s - 

1273/1273 [==============================] - 0s - loss: 2.0655 - acc: 0.9796     
Epoch 127/200
1273/1273 [==============================] - 0s - loss: 2.0460 - acc: 0.9874     
Epoch 128/200
1273/1273 [==============================] - 0s - loss: 2.0486 - acc: 0.9835     
Epoch 129/200
1273/1273 [==============================] - 0s - loss: 2.0378 - acc: 0.9851     
Epoch 130/200
1273/1273 [==============================] - 0s - loss: 2.0357 - acc: 0.9827     
Epoch 131/200
1273/1273 [==============================] - 0s - loss: 2.0395 - acc: 0.9796     
Epoch 132/200
1273/1273 [==============================] - 0s - loss: 2.0285 - acc: 0.9788     
Epoch 133/200
1273/1273 [==============================] - 0s - loss: 2.0237 - acc: 0.9804     
Epoch 134/200
1273/1273 [==============================] - 0s - loss: 2.0147 - acc: 0.9764     
Epoch 135/200
1273/1273 [==============================] - 0s - loss: 2.0176 - acc: 0.9788     
Epoch 136/200
1273/1273 [=============================

1273/1273 [==============================] - 0s - loss: 2.3393 - acc: 0.9749     
Epoch 96/200
1273/1273 [==============================] - 0s - loss: 2.3274 - acc: 0.9764     
Epoch 97/200
1273/1273 [==============================] - 0s - loss: 2.3152 - acc: 0.9780     
Epoch 98/200
1273/1273 [==============================] - 0s - loss: 2.3178 - acc: 0.9749     
Epoch 99/200
1273/1273 [==============================] - 0s - loss: 2.2983 - acc: 0.9819     
Epoch 100/200
1273/1273 [==============================] - 0s - loss: 2.2920 - acc: 0.9796     
Epoch 101/200
1273/1273 [==============================] - 0s - loss: 2.2948 - acc: 0.9678     
Epoch 102/200
1273/1273 [==============================] - 0s - loss: 2.2743 - acc: 0.9772     
Epoch 103/200
1273/1273 [==============================] - 0s - loss: 2.2680 - acc: 0.9780     
Epoch 104/200
1273/1273 [==============================] - 0s - loss: 2.2594 - acc: 0.9811     
Epoch 105/200
1273/1273 [==============================] -

1273/1273 [==============================] - 0s - loss: 2.6535 - acc: 0.9788     
Epoch 65/200
1273/1273 [==============================] - 0s - loss: 2.6357 - acc: 0.9811     
Epoch 66/200
1273/1273 [==============================] - 0s - loss: 2.6161 - acc: 0.9772     
Epoch 67/200
1273/1273 [==============================] - 0s - loss: 2.6040 - acc: 0.9804     
Epoch 68/200
1273/1273 [==============================] - 0s - loss: 2.6001 - acc: 0.9709     
Epoch 69/200
1273/1273 [==============================] - 0s - loss: 2.5773 - acc: 0.9835     
Epoch 70/200
1273/1273 [==============================] - 0s - loss: 2.5662 - acc: 0.9772     
Epoch 71/200
1273/1273 [==============================] - 0s - loss: 2.5536 - acc: 0.9741     
Epoch 72/200
1273/1273 [==============================] - 0s - loss: 2.5343 - acc: 0.9764     
Epoch 73/200
1273/1273 [==============================] - 0s - loss: 2.5417 - acc: 0.9701     
Epoch 74/200
1273/1273 [==============================] - 0s - 

# 'known' train, 'unknown' test

In [3]:
train_tactile_images = np.load('images-known-t3-cero2lesscontact-stdnorm.npy')
train_labels = np.load('labels-known-t3-cero2lesscontact-stdnorm.npy')
train_labels_cat = np_utils.to_categorical(train_labels, num_classes=2)

test_tactile_images = np.load('images-unknown-t3-cero2lesscontact-stdnorm.npy')
test_labels = np.load('labels-unknown-t3-cero2lesscontact-stdnorm.npy')
test_labels_cat = np_utils.to_categorical(test_labels, num_classes=2)

FINGERS = 3
TACTILE_IMAGE_ROWS = 6
TACTILE_IMAGE_COLS = 7

print train_tactile_images.shape
print test_tactile_images.shape

(1931, 3, 6, 7)
(618, 3, 6, 7)


In [13]:
# BEST

folds = 10
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for fold in range(folds):
    print "Training fold " + str(fold + 1) + "/" + str(folds)
    
    shuffle(train_tactile_images, train_labels_cat)
    shuffle(test_tactile_images, test_labels_cat)
    
    # build model
    epochs = 200
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.005
    
    l2_reg = 0.05
    drop_prob = 0.2
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(train_tactile_images, train_labels_cat, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print "# # # Evaluating cv-fold..."
    scores = model.evaluate(test_tactile_images, test_labels_cat, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(test_tactile_images)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(test_labels_cat, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print "\naccuracy:", scores[1] * 100
    print "precision:", precision * 100
    print "recall:", recall * 100
    print "f1_score:", f1_score * 100
    
print "\nFinal cross-validation score:"
print np.mean(cv_accuracies), "+/-", np.std(cv_accuracies)
print np.mean(cv_precision), "+/-", np.std(cv_precision)
print np.mean(cv_recall), "+/-", np.std(cv_recall)
print np.mean(cv_f1_score), "+/-", np.std(cv_f1_score)

Training fold 1/10
Epoch 1/200
1931/1931 [==============================] - 1s - loss: 36.3331 - acc: 0.7975     
Epoch 2/200
1931/1931 [==============================] - 0s - loss: 29.8283 - acc: 0.8633     
Epoch 3/200
1931/1931 [==============================] - 0s - loss: 25.3991 - acc: 0.8819     
Epoch 4/200
1931/1931 [==============================] - 0s - loss: 22.1544 - acc: 0.8933     
Epoch 5/200
1931/1931 [==============================] - 0s - loss: 19.6752 - acc: 0.9006     
Epoch 6/200
1931/1931 [==============================] - 0s - loss: 17.7095 - acc: 0.9104     
Epoch 7/200
1931/1931 [==============================] - 0s - loss: 16.1267 - acc: 0.9057     
Epoch 8/200
1931/1931 [==============================] - 0s - loss: 14.7994 - acc: 0.9208     
Epoch 9/200
1931/1931 [==============================] - 0s - loss: 13.6954 - acc: 0.9140     
Epoch 10/200
1931/1931 [==============================] - 0s - loss: 12.7325 - acc: 0.9249     
Epoch 11/200
1931/1931 [======

1931/1931 [==============================] - 0s - loss: 1.7041 - acc: 0.9788     
Epoch 172/200
1931/1931 [==============================] - 0s - loss: 1.6958 - acc: 0.9788     
Epoch 173/200
1931/1931 [==============================] - 0s - loss: 1.7002 - acc: 0.9736     
Epoch 174/200
1931/1931 [==============================] - 0s - loss: 1.6913 - acc: 0.9767     
Epoch 175/200
1931/1931 [==============================] - 0s - loss: 1.6924 - acc: 0.9757     
Epoch 176/200
1931/1931 [==============================] - 0s - loss: 1.6800 - acc: 0.9814     
Epoch 177/200
1931/1931 [==============================] - 0s - loss: 1.6789 - acc: 0.9757     
Epoch 178/200
1931/1931 [==============================] - 0s - loss: 1.6733 - acc: 0.9736     
Epoch 179/200
1931/1931 [==============================] - 0s - loss: 1.6726 - acc: 0.9746     
Epoch 180/200
1931/1931 [==============================] - 0s - loss: 1.6645 - acc: 0.9814     
Epoch 181/200
1931/1931 [=============================

1931/1931 [==============================] - 0s - loss: 1.9198 - acc: 0.9757     
Epoch 140/200
1931/1931 [==============================] - 0s - loss: 1.9097 - acc: 0.9710     
Epoch 141/200
1931/1931 [==============================] - 0s - loss: 1.9058 - acc: 0.9762     
Epoch 142/200
1931/1931 [==============================] - 0s - loss: 1.9027 - acc: 0.9720     
Epoch 143/200
1931/1931 [==============================] - 0s - loss: 1.8957 - acc: 0.9700     
Epoch 144/200
1931/1931 [==============================] - 0s - loss: 1.8829 - acc: 0.9772     
Epoch 145/200
1931/1931 [==============================] - 0s - loss: 1.8821 - acc: 0.9736     
Epoch 146/200
1931/1931 [==============================] - 0s - loss: 1.8761 - acc: 0.9736     
Epoch 147/200
1931/1931 [==============================] - 0s - loss: 1.8681 - acc: 0.9746     
Epoch 148/200
1931/1931 [==============================] - 0s - loss: 1.8604 - acc: 0.9782     
Epoch 149/200
1931/1931 [=============================

1931/1931 [==============================] - 0s - loss: 2.1920 - acc: 0.9731     
Epoch 109/200
1931/1931 [==============================] - 0s - loss: 2.1756 - acc: 0.9746     
Epoch 110/200
1931/1931 [==============================] - 0s - loss: 2.1673 - acc: 0.9715     
Epoch 111/200
1931/1931 [==============================] - 0s - loss: 2.1569 - acc: 0.9715     
Epoch 112/200
1931/1931 [==============================] - 0s - loss: 2.1415 - acc: 0.9746     
Epoch 113/200
1931/1931 [==============================] - 0s - loss: 2.1327 - acc: 0.9741     
Epoch 114/200
1931/1931 [==============================] - 0s - loss: 2.1241 - acc: 0.9746     
Epoch 115/200
1931/1931 [==============================] - 0s - loss: 2.1153 - acc: 0.9684     
Epoch 116/200
1931/1931 [==============================] - 0s - loss: 2.1016 - acc: 0.9736     
Epoch 117/200
1931/1931 [==============================] - 0s - loss: 2.1027 - acc: 0.9669     
Epoch 118/200
1931/1931 [=============================

1931/1931 [==============================] - 0s - loss: 2.6108 - acc: 0.9632     
Epoch 78/200
1931/1931 [==============================] - 0s - loss: 2.5894 - acc: 0.9679     
Epoch 79/200
1931/1931 [==============================] - 0s - loss: 2.5626 - acc: 0.9700     
Epoch 80/200
1931/1931 [==============================] - 0s - loss: 2.5436 - acc: 0.9674     
Epoch 81/200
1931/1931 [==============================] - 0s - loss: 2.5241 - acc: 0.9669     
Epoch 82/200
1931/1931 [==============================] - 0s - loss: 2.5037 - acc: 0.9632     
Epoch 83/200
1931/1931 [==============================] - 0s - loss: 2.4812 - acc: 0.9684     
Epoch 84/200
1931/1931 [==============================] - 0s - loss: 2.4633 - acc: 0.9663     
Epoch 85/200
1931/1931 [==============================] - 0s - loss: 2.4396 - acc: 0.9720     
Epoch 86/200
1931/1931 [==============================] - 0s - loss: 2.4275 - acc: 0.9669     
Epoch 87/200
1931/1931 [==============================] - 0s - 

...

1931/1931 [==============================] - 0s - loss: 2.9222 - acc: 0.9617     
Epoch 68/200
1931/1931 [==============================] - 0s - loss: 2.8803 - acc: 0.9653     
Epoch 69/200
1931/1931 [==============================] - 0s - loss: 2.8580 - acc: 0.9694     
Epoch 70/200
1931/1931 [==============================] - 0s - loss: 2.8315 - acc: 0.9637     
Epoch 71/200
1931/1931 [==============================] - 0s - loss: 2.8002 - acc: 0.9663     
Epoch 72/200
1931/1931 [==============================] - 0s - loss: 2.7774 - acc: 0.9606     
Epoch 73/200
1931/1931 [==============================] - 0s - loss: 2.7527 - acc: 0.9648     
Epoch 74/200
1931/1931 [==============================] - 0s - loss: 2.7210 - acc: 0.9694     
Epoch 75/200
1931/1931 [==============================] - 0s - loss: 2.6939 - acc: 0.9705     
Epoch 76/200
1931/1931 [==============================] - 0s - loss: 2.6817 - acc: 0.9643     
Epoch 77/200
1931/1931 [==============================] - 0s - 

1931/1931 [==============================] - 0s - loss: 4.9090 - acc: 0.9570     
Epoch 36/200
1931/1931 [==============================] - 0s - loss: 4.8019 - acc: 0.9503     
Epoch 37/200
1931/1931 [==============================] - 0s - loss: 4.6958 - acc: 0.9539     
Epoch 38/200
1931/1931 [==============================] - 0s - loss: 4.5955 - acc: 0.9575     
Epoch 39/200
1931/1931 [==============================] - 0s - loss: 4.5018 - acc: 0.9560     
Epoch 40/200
1931/1931 [==============================] - 0s - loss: 4.4177 - acc: 0.9508     
Epoch 41/200
1931/1931 [==============================] - 0s - loss: 4.3287 - acc: 0.9492     
Epoch 42/200
1931/1931 [==============================] - 0s - loss: 4.2468 - acc: 0.9581     
Epoch 43/200
1931/1931 [==============================] - 0s - loss: 4.1642 - acc: 0.9586     
Epoch 44/200
1931/1931 [==============================] - 0s - loss: 4.0978 - acc: 0.9570     
Epoch 45/200
1931/1931 [==============================] - 0s - 

1931/1931 [==============================] - 0s - loss: 22.1075 - acc: 0.8969     
Epoch 5/200
1931/1931 [==============================] - 0s - loss: 19.6719 - acc: 0.8938     
Epoch 6/200
1931/1931 [==============================] - 0s - loss: 17.7234 - acc: 0.9052     
Epoch 7/200
1931/1931 [==============================] - 0s - loss: 16.1443 - acc: 0.9104     
Epoch 8/200
1931/1931 [==============================] - 0s - loss: 14.8454 - acc: 0.9171     
Epoch 9/200
1931/1931 [==============================] - 0s - loss: 13.7338 - acc: 0.9254     
Epoch 10/200
1931/1931 [==============================] - 0s - loss: 12.7942 - acc: 0.9228     
Epoch 11/200
1931/1931 [==============================] - 0s - loss: 11.9707 - acc: 0.9316     
Epoch 12/200
1931/1931 [==============================] - 0s - loss: 11.2583 - acc: 0.9301     
Epoch 13/200
1931/1931 [==============================] - 0s - loss: 10.6177 - acc: 0.9322     
Epoch 14/200
1931/1931 [==============================] - 

1931/1931 [==============================] - 0s - loss: 1.7210 - acc: 0.9777     
Epoch 176/200
1931/1931 [==============================] - 0s - loss: 1.7074 - acc: 0.9793     
Epoch 177/200
1931/1931 [==============================] - 0s - loss: 1.7075 - acc: 0.9819     
Epoch 178/200
1931/1931 [==============================] - 0s - loss: 1.7000 - acc: 0.9798     
Epoch 179/200
1931/1931 [==============================] - 0s - loss: 1.6938 - acc: 0.9793     
Epoch 180/200
1931/1931 [==============================] - 0s - loss: 1.6932 - acc: 0.9782     
Epoch 181/200
1931/1931 [==============================] - 0s - loss: 1.6913 - acc: 0.9777     
Epoch 182/200
1931/1931 [==============================] - 0s - loss: 1.6837 - acc: 0.9798     
Epoch 183/200
1931/1931 [==============================] - 0s - loss: 1.6765 - acc: 0.9819     
Epoch 184/200
1931/1931 [==============================] - 0s - loss: 1.6838 - acc: 0.9736     
Epoch 185/200
1931/1931 [=============================

1931/1931 [==============================] - 0s - loss: 1.8900 - acc: 0.9741     
Epoch 144/200
1931/1931 [==============================] - 0s - loss: 1.8756 - acc: 0.9798     
Epoch 145/200
1931/1931 [==============================] - 0s - loss: 1.8783 - acc: 0.9746     
Epoch 146/200
1931/1931 [==============================] - 0s - loss: 1.8663 - acc: 0.9772     
Epoch 147/200
1931/1931 [==============================] - 0s - loss: 1.8536 - acc: 0.9757     
Epoch 148/200
1931/1931 [==============================] - 0s - loss: 1.8585 - acc: 0.9762     
Epoch 149/200
1931/1931 [==============================] - 0s - loss: 1.8500 - acc: 0.9762     
Epoch 150/200
1931/1931 [==============================] - 0s - loss: 1.8420 - acc: 0.9741     
Epoch 151/200
1931/1931 [==============================] - 0s - loss: 1.8357 - acc: 0.9767     
Epoch 152/200
1931/1931 [==============================] - 0s - loss: 1.8316 - acc: 0.9782     
Epoch 153/200
1931/1931 [=============================